# 🔍 Africa Gender & Youth Indicators
## Notebook 02: Exploratory Analysis & SQL Queries

**Author:** Teresiah Njoroge
**Date:** February 2026
**Project:** AU WGYD M&E Data Pipeline

---

### What This Notebook Does
We load the 5 cleaned indicator datasets into a SQL database
(SQLite — a lightweight database that runs locally) and run
analytical queries to answer key M&E questions:

| Query | Question |
|---|---|
| Q1 | Where does each country stand right now? |
| Q2 | Is the continental average improving over time? |
| Q3 | Which regions are leading and which are lagging? |
| Q4 | Who are the top and bottom performers? |
| Q5 | What is the gender gap in youth unemployment? |
| Q6 | Where are the data gaps by region? |

> **Why SQL in Python?**
> Programme staff at the AU ask specific questions on demand.
> SQL lets us answer those questions quickly and reproducibly
> without manually filtering spreadsheets every time.


In [3]:
import pandas as pd
import numpy as np
import sqlite3
import os
import warnings
warnings.filterwarnings('ignore')

# Go UP one level from current notebook location to find 01_data/cleaned
notebook_dir = os.getcwd()
print("📁 Notebook is running from:", notebook_dir)

# Check if 'cleaned' folder is directly here or one level up
if os.path.isdir(os.path.join(notebook_dir, 'cleaned')):
    # Notebook is INSIDE 01_data folder
    CLEAN = os.path.join(notebook_dir, 'cleaned')
elif os.path.isdir(os.path.join(notebook_dir, '01_data', 'cleaned')):
    # Notebook is OUTSIDE 01_data folder
    CLEAN = os.path.join(notebook_dir, '01_data', 'cleaned')
else:
    print("❌ Cannot find cleaned folder — check your folder structure")
    CLEAN = None

print("✅ Libraries imported")
print("📁 CLEAN folder set to:", CLEAN)
print("📁 Folder exists:", os.path.isdir(CLEAN) if CLEAN else False)



📁 Notebook is running from: c:\Users\wanji\Downloads\TEE\Doing Data Analysis\Tasks\01_data
✅ Libraries imported
📁 CLEAN folder set to: c:\Users\wanji\Downloads\TEE\Doing Data Analysis\Tasks\01_data\cleaned
📁 Folder exists: True


## Step 1: Load Cleaned Data into a SQL Database

We load all 5 cleaned CSV files into a SQLite database.

**What is SQLite?**
Think of it as a mini-database that lives inside your computer
— no server needed, no installation needed.
It works exactly like PostgreSQL or MySQL but runs locally.

**What is a database table?**
Each CSV file becomes one table in the database.
A table is just like an Excel sheet — rows and columns.
But unlike Excel, we can now ask it questions using SQL.

After this step our database has one table called
`cleaned_indicators` containing ALL 5 indicators combined —
2,430 rows total (54 countries × 9 years × 5 indicators).


In [4]:
# Step 1: Load all 5 cleaned CSVs into one combined dataframe
all_frames = []

for f in os.listdir(CLEAN):
    if f.endswith('_clean.csv'):
        path = os.path.join(CLEAN, f)
        df   = pd.read_csv(path)
        all_frames.append(df)
        print(f"✅ Loaded: {f} | Rows: {len(df)}")

# Combine all 5 into one single table
combined = pd.concat(all_frames, ignore_index=True)
print(f"\n📊 Combined table shape: {combined.shape}")
print(f"   Indicators: {combined['indicator'].unique().tolist()}")

# Step 2: Load into SQLite database (lives in memory)
conn = sqlite3.connect(':memory:')   # :memory: means no file saved — runs in RAM

# Write the combined dataframe as a SQL table called 'cleaned_indicators'
combined.to_sql('cleaned_indicators', conn, if_exists='replace', index=False)

print(f"\n✅ Database ready — table 'cleaned_indicators' loaded")
print(f"   Total rows in database: {pd.read_sql('SELECT COUNT(*) AS total FROM cleaned_indicators', conn).iloc[0,0]}")


✅ Loaded: female_lfp_clean.csv | Rows: 486
✅ Loaded: girls_secondary_enrol_clean.csv | Rows: 486
✅ Loaded: women_in_parliament_clean.csv | Rows: 486
✅ Loaded: youth_unemployment_f_clean.csv | Rows: 486
✅ Loaded: youth_unemployment_m_clean.csv | Rows: 486

📊 Combined table shape: (2430, 6)
   Indicators: ['female_lfp', 'girls_secondary_enrol', 'women_in_parliament', 'youth_unemployment_f', 'youth_unemployment_m']

✅ Database ready — table 'cleaned_indicators' loaded
   Total rows in database: 2430


## Query 1: Where does each country stand right now?

**The question:** What is the latest available value for every
country for every indicator?

**Why this matters for WGYD:**
Before tracking progress, you need a baseline snapshot —
where is each country TODAY? This is the first table that
goes into any M&E dashboard or programme brief.

**How the SQL works:**
We use a subquery — a query inside a query.
The inner query finds the most recent year for each
country-indicator combination.
The outer query then fetches the actual value for that year.


In [9]:
query1 = """
SELECT
    country_code,
    au_region,
    indicator,
    year        AS latest_year,
    ROUND(value, 2) AS latest_value,
    quality_flag
FROM cleaned_indicators ci
WHERE quality_flag = 'VERIFIED'
AND year = (
    SELECT MAX(year)
    FROM cleaned_indicators
    WHERE country_code = ci.country_code
    AND   indicator    = ci.indicator
    AND   quality_flag = 'VERIFIED'
)
ORDER BY indicator, au_region, country_code
"""

# Run the query and load result into a pandas dataframe
q1_result = pd.read_sql(query1, conn)

print(f"✅ Query 1 complete")
print(f"   Rows returned: {len(q1_result)}")
print(f"   Columns: {q1_result.columns.tolist()}")
print()

# Show first 10 rows as a preview
print("📋 Preview — first 10 rows:")
print(q1_result.head(10).to_string(index=False))


✅ Query 1 complete
   Rows returned: 258
   Columns: ['country_code', 'au_region', 'indicator', 'latest_year', 'latest_value', 'quality_flag']

📋 Preview — first 10 rows:
country_code      au_region  indicator  latest_year  latest_value quality_flag
         AGO Central Africa female_lfp         2023         73.39     VERIFIED
         CAF Central Africa female_lfp         2023         66.21     VERIFIED
         CMR Central Africa female_lfp         2023         57.66     VERIFIED
         COD Central Africa female_lfp         2023         62.17     VERIFIED
         COG Central Africa female_lfp         2023         66.60     VERIFIED
         GAB Central Africa female_lfp         2023         43.62     VERIFIED
         GNQ Central Africa female_lfp         2023         55.11     VERIFIED
         STP Central Africa female_lfp         2023         22.87     VERIFIED
         TCD Central Africa female_lfp         2023         48.02     VERIFIED
         BDI    East Africa female_lfp 

## Query 2: Is Africa improving over time?

**The question:** What is the continental average for each
indicator per year from 2015 to 2023?

**Why this matters for WGYD:**
This is the core trend question in any M&E framework.
Are we moving in the right direction?
Are we on track to meet 2030 targets?
This query produces the numbers behind trend line charts.

**How the SQL works:**
We group all countries together by indicator and year,
then calculate the average value across all countries
that reported data that year.
We also calculate data completeness — how many countries
actually reported vs how many should have.


In [10]:
query2 = """
SELECT
    indicator,
    year,
    ROUND(AVG(value), 2)                        AS continental_avg,
    COUNT(value)                                AS countries_reporting,
    COUNT(*)                                    AS total_countries,
    ROUND(COUNT(value) * 100.0 / COUNT(*), 1)   AS completeness_pct
FROM cleaned_indicators
WHERE quality_flag = 'VERIFIED'
GROUP BY indicator, year
ORDER BY indicator, year
"""

q2_result = pd.read_sql(query2, conn)

print("✅ Query 2 complete")
print(f"   Rows returned: {len(q2_result)}")
print()

# Show each indicator as its own mini table
for indicator in q2_result['indicator'].unique():
    df_ind = q2_result[q2_result['indicator'] == indicator].copy()
    print(f"📊 {indicator.upper().replace('_',' ')}")
    print(df_ind[['year','continental_avg',
                  'countries_reporting','completeness_pct']].to_string(index=False))
    print()


✅ Query 2 complete
   Rows returned: 45

📊 FEMALE LFP
 year  continental_avg  countries_reporting  completeness_pct
 2015            55.31                   53             100.0
 2016            55.12                   53             100.0
 2017            54.74                   53             100.0
 2018            54.41                   53             100.0
 2019            54.30                   53             100.0
 2020            53.29                   53             100.0
 2021            53.67                   53             100.0
 2022            54.22                   53             100.0
 2023            55.52                   52             100.0

📊 GIRLS SECONDARY ENROL
 year  continental_avg  countries_reporting  completeness_pct
 2015            49.89                   32             100.0
 2016            55.87                   30             100.0
 2017            55.22                   28             100.0
 2018            56.82                   23          

## Query 3: Which regions are leading and which are lagging?

**The question:** What is the average value per AU region
for each indicator in the most recent year?

**Why this matters for WGYD:**
The AU works across 5 regions. A continental average
hides regional inequality — one region doing very well
can mask another doing very poorly.

WGYD programme teams use regional breakdowns to:
- Target interventions where gaps are largest
- Report to member states on regional standing
- Design region-specific capacity building programmes

**How the SQL works:**
We filter to verified data only, group by region and
indicator, then sort from highest to lowest average.
This immediately shows which regions lead and which
need the most support.


In [11]:
query3 = """
SELECT
    indicator,
    au_region,
    ROUND(AVG(value), 2)    AS regional_avg,
    COUNT(value)            AS countries_reporting,
    MAX(year)               AS latest_year
FROM cleaned_indicators
WHERE quality_flag = 'VERIFIED'
GROUP BY indicator, au_region
ORDER BY indicator, regional_avg DESC
"""

q3_result = pd.read_sql(query3, conn)

print("✅ Query 3 complete")
print(f"   Rows returned: {len(q3_result)}")
print()

# Show each indicator as its own regional breakdown table
for indicator in q3_result['indicator'].unique():
    df_ind = q3_result[q3_result['indicator'] == indicator].copy()
    print(f"📊 {indicator.upper().replace('_', ' ')} — Regional Ranking")
    print(f"   {'Region':<20} {'Avg':>8} {'Countries':>12} {'Latest Year':>12}")
    print(f"   {'-'*54}")
    for _, row in df_ind.iterrows():
        print(f"   {row['au_region']:<20} "
              f"{row['regional_avg']:>8.1f} "
              f"{int(row['countries_reporting']):>12} "
              f"{int(row['latest_year']):>12}")
    print()


✅ Query 3 complete
   Rows returned: 29

📊 FEMALE LFP — Regional Ranking
   Region                    Avg    Countries  Latest Year
   ------------------------------------------------------
   East Africa              61.2          117         2023
   Southern Africa          58.4           81         2023
   West Africa              57.6          144         2023
   Central Africa           55.5           81         2023
   Other                    35.8            9         2023
   North Africa             21.7           44         2023

📊 GIRLS SECONDARY ENROL — Regional Ranking
   Region                    Avg    Countries  Latest Year
   ------------------------------------------------------
   Southern Africa          74.0           36         2023
   North Africa             73.5           22         2023
   East Africa              57.5           56         2023
   West Africa              52.5           79         2023
   Central Africa           40.3           23         2023


## Query 4: Who are the top and bottom performers?

**The question:** For women in parliament — which 5 countries
are leading and which 5 need the most support?

**Why this matters for WGYD:**
Top performers can be highlighted as models for peer learning.
Bottom performers need targeted policy support and
capacity building. This is how the AU designs
country-specific interventions.

**How the SQL works:**
We use UNION ALL to combine two separate queries
into one result table.
- First query: sorts highest to lowest, takes top 5
- Second query: sorts lowest to highest, takes bottom 5
UNION ALL stacks them on top of each other.
We add a label column called 'category' so we know
which group each country belongs to.


In [18]:
query4 = """
SELECT * FROM (
    SELECT
        country_code,
        au_region,
        year,
        ROUND(value, 2)     AS women_in_parliament_pct,
        'Top Performer'     AS category
    FROM cleaned_indicators
    WHERE indicator    = 'women_in_parliament'
    AND   quality_flag = 'VERIFIED'
    AND   year = (
        SELECT MAX(year)
        FROM cleaned_indicators
        WHERE indicator    = 'women_in_parliament'
        AND   quality_flag = 'VERIFIED'
    )
    ORDER BY value DESC
    LIMIT 5
)

UNION ALL

SELECT * FROM (
    SELECT
        country_code,
        au_region,
        year,
        ROUND(value, 2)     AS women_in_parliament_pct,
        'Needs Support'     AS category
    FROM cleaned_indicators
    WHERE indicator    = 'women_in_parliament'
    AND   quality_flag = 'VERIFIED'
    AND   year = (
        SELECT MAX(year)
        FROM cleaned_indicators
        WHERE indicator    = 'women_in_parliament'
        AND   quality_flag = 'VERIFIED'
    )
    ORDER BY value ASC
    LIMIT 5
)
"""

q4_result = pd.read_sql(query4, conn)

print(" Query 4 complete")
print()

# Top performers
top = q4_result[q4_result['category'] == 'Top Performer']
print(" TOP 5 — Women in Parliament")
print(f"   {'Country':<10} {'Region':<20} {'% Women':>10} {'Year':>6}")
print(f"   {'-'*50}")
for _, row in top.iterrows():
    print(f"   {row['country_code']:<10} "
          f"{row['au_region']:<20} "
          f"{row['women_in_parliament_pct']:>10.1f} "
          f"{int(row['year']):>6}")

print()

# Bottom performers
bottom = q4_result[q4_result['category'] == 'Needs Support']
print(" BOTTOM 5 — Women in Parliament (Need Support)")
print(f"   {'Country':<10} {'Region':<20} {'% Women':>10} {'Year':>6}")
print(f"   {'-'*50}")
for _, row in bottom.iterrows():
    print(f"   {row['country_code']:<10} "
          f"{row['au_region']:<20} "
          f"{row['women_in_parliament_pct']:>10.1f} "
          f"{int(row['year']):>6}")


 Query 4 complete

 TOP 5 — Women in Parliament
   Country    Region                  % Women   Year
   --------------------------------------------------
   RWA        East Africa                61.2   2023
   SEN        West Africa                46.1   2023
   ZAF        Southern Africa            45.9   2023
   NAM        Southern Africa            44.2   2023
   MOZ        Southern Africa            43.2   2023

 BOTTOM 5 — Women in Parliament (Need Support)
   Country    Region                  % Women   Year
   --------------------------------------------------
   NGA        West Africa                 3.9   2023
   DZA        North Africa                7.9   2023
   GMB        West Africa                 8.6   2023
   GNB        West Africa                 9.8   2023
   LBR        West Africa                11.0   2023


## Query 5: What is the gender gap in youth unemployment?

**The question:** For each country, how much higher or lower
is female youth unemployment compared to male?

**Why this matters for WGYD:**
A country may have "low unemployment" overall but hide a
large gender gap underneath. If young women are unemployed
at twice the rate of young men, that is a policy emergency
but it only becomes visible when you compare the two.

This query directly supports the AU's youth empowerment
mandate by identifying where young women are most
economically excluded.

**How the SQL works:**
We JOIN two copies of the same table together.
- First copy = female unemployment rows only
- Second copy = male unemployment rows only
We match them on country AND year, then subtract
one value from the other to calculate the gap.
A CASE statement labels which gender is disadvantaged.


In [19]:
query5 = """
SELECT
    f.country_code,
    f.au_region,
    f.year,
    ROUND(f.value, 2)           AS female_unemployment,
    ROUND(m.value, 2)           AS male_unemployment,
    ROUND(f.value - m.value, 2) AS gender_gap,
    CASE
        WHEN f.value > m.value THEN 'Female disadvantaged'
        WHEN f.value < m.value THEN 'Male disadvantaged'
        ELSE 'Parity'
    END                         AS gap_direction
FROM cleaned_indicators f
JOIN cleaned_indicators m
    ON  f.country_code = m.country_code
    AND f.year         = m.year
WHERE f.indicator    = 'youth_unemployment_f'
AND   m.indicator    = 'youth_unemployment_m'
AND   f.quality_flag = 'VERIFIED'
AND   m.quality_flag = 'VERIFIED'
AND   f.year = (
    SELECT MAX(year)
    FROM cleaned_indicators
    WHERE indicator    = 'youth_unemployment_f'
    AND   quality_flag = 'VERIFIED'
)
ORDER BY ABS(f.value - m.value) DESC
"""

q5_result = pd.read_sql(query5, conn)

print("✅ Query 5 complete")
print(f"   Countries with data: {len(q5_result)}")
print()

# Summary counts
female_disadv = (q5_result['gap_direction'] == 'Female disadvantaged').sum()
male_disadv   = (q5_result['gap_direction'] == 'Male disadvantaged').sum()
parity        = (q5_result['gap_direction'] == 'Parity').sum()

print("📊 Gender Gap Summary — Youth Unemployment")
print(f"   Female disadvantaged: {female_disadv} countries")
print(f"   Male disadvantaged:   {male_disadv} countries")
print(f"   Parity:               {parity} countries")
print()

# Top 10 largest gaps
print("🔍 Top 10 Largest Gender Gaps:")
print(f"   {'Country':<10} {'Region':<20} {'Female':>8} "
      f"{'Male':>8} {'Gap':>8} {'Direction':<22}")
print(f"   {'-'*78}")
for _, row in q5_result.head(10).iterrows():
    print(f"   {row['country_code']:<10} "
          f"{row['au_region']:<20} "
          f"{row['female_unemployment']:>8.1f} "
          f"{row['male_unemployment']:>8.1f} "
          f"{row['gender_gap']:>8.1f} "
          f"{row['gap_direction']:<22}")


✅ Query 5 complete
   Countries with data: 52

📊 Gender Gap Summary — Youth Unemployment
   Female disadvantaged: 36 countries
   Male disadvantaged:   16 countries
   Parity:               0 countries

🔍 Top 10 Largest Gender Gaps:
   Country    Region                 Female     Male      Gap Direction             
   ------------------------------------------------------------------------------
   LBY        Other                    68.7     41.9     26.8 Female disadvantaged  
   EGY        North Africa             36.6     12.0     24.6 Female disadvantaged  
   LSO        Southern Africa          37.2     18.1     19.1 Female disadvantaged  
   DZA        North Africa             45.7     27.1     18.7 Female disadvantaged  
   GAB        Central Africa           42.6     31.7     11.0 Female disadvantaged  
   MRT        West Africa              30.3     19.9     10.4 Female disadvantaged  
   BWA        Southern Africa          50.0     39.6     10.4 Female disadvantaged  
   CP

## Query 6: Where are the data gaps by region?

**The question:** For each indicator and AU region, what
percentage of expected data points are actually available?

**Why this matters for WGYD:**
Before publishing any report or dashboard, you must
disclose data gaps to stakeholders. A number on a
dashboard without a completeness score is misleading.

This query produces the data quality table that goes
directly into programme reports — it shows leadership
exactly where data collection systems need strengthening.

A completeness below 80% means that region's average
should NOT be used for policy decisions without a caveat.

**How the SQL works:**
We use CASE WHEN inside SUM to count specific values.
This is called conditional aggregation — counting only
rows that meet a condition inside a summary calculation.


In [20]:
query6 = """
SELECT
    indicator,
    au_region,
    COUNT(*)                                                       AS total_records,
    SUM(CASE WHEN quality_flag = 'VERIFIED' THEN 1 ELSE 0 END)    AS verified,
    SUM(CASE WHEN quality_flag = 'MISSING'  THEN 1 ELSE 0 END)    AS missing,
    ROUND(
        SUM(CASE WHEN quality_flag = 'VERIFIED' THEN 1 ELSE 0 END)
        * 100.0 / COUNT(*), 1
    )                                                              AS completeness_pct
FROM cleaned_indicators
GROUP BY indicator, au_region
ORDER BY indicator, completeness_pct ASC
"""

q6_result = pd.read_sql(query6, conn)

print("✅ Query 6 complete")
print()

# Show each indicator separately
for indicator in q6_result['indicator'].unique():
    df_ind = q6_result[q6_result['indicator'] == indicator].copy()

    print(f"📊 {indicator.upper().replace('_', ' ')}")
    print(f"   {'Region':<22} {'Total':>7} {'Verified':>10} "
          f"{'Missing':>9} {'Complete%':>10} {'Status':>10}")
    print(f"   {'-'*72}")

    for _, row in df_ind.iterrows():
        # Traffic light status
        if row['completeness_pct'] >= 80:
            status = '✅ Good'
        elif row['completeness_pct'] >= 50:
            status = '⚠️  Fair'
        else:
            status = '🔴 Poor'

        print(f"   {row['au_region']:<22} "
              f"{int(row['total_records']):>7} "
              f"{int(row['verified']):>10} "
              f"{int(row['missing']):>9} "
              f"{row['completeness_pct']:>9.1f}% "
              f"{status:>10}")
    print()


✅ Query 6 complete

📊 FEMALE LFP
   Region                   Total   Verified   Missing  Complete%     Status
   ------------------------------------------------------------------------
   East Africa                126        117         9      92.9%     ✅ Good
   North Africa                45         44         1      97.8%     ✅ Good
   Central Africa              81         81         0     100.0%     ✅ Good
   Other                        9          9         0     100.0%     ✅ Good
   Southern Africa             81         81         0     100.0%     ✅ Good
   West Africa                144        144         0     100.0%     ✅ Good

📊 GIRLS SECONDARY ENROL
   Region                   Total   Verified   Missing  Complete%     Status
   ------------------------------------------------------------------------
   Other                        9          0         9       0.0%     🔴 Poor
   Central Africa              81         23        58      28.4%     🔴 Poor
   East Africa      

## Notebook 02 Summary — Key Findings

This section summarises the analytical findings from all
6 queries. These findings are written in plain English
so they can be copied directly into a programme brief
or dashboard annotation for non-technical audiences.

> **This is what "translating data for non-technical
> audiences" looks like in practice.**


In [21]:
print("=" * 60)
print("AFRICA GENDER & YOUTH INDICATORS — KEY FINDINGS")
print("AU WGYD M&E Data Pipeline | Teresiah Njoroge | Feb 2026")
print("=" * 60)

# Finding 1: Women in parliament top performer
top_country = q4_result[q4_result['category'] == 'Top Performer'].iloc[0]
bottom_country = q4_result[q4_result['category'] == 'Needs Support'].iloc[0]

print("""
FINDING 1 — WOMEN IN PARLIAMENT
--------------------------------
The continent shows significant variation in female
political representation across AU member states.
The leading country significantly outperforms the
continental average, while the bottom performers
remain far below the AU's gender parity targets.
""")
print(f"  Top performer:    {top_country['country_code']} "
      f"({top_country['au_region']}) — "
      f"{top_country['women_in_parliament_pct']}%")
print(f"  Lowest performer: {bottom_country['country_code']} "
      f"({bottom_country['au_region']}) — "
      f"{bottom_country['women_in_parliament_pct']}%")

# Finding 2: Gender gap in unemployment
female_disadv = (q5_result['gap_direction'] == 'Female disadvantaged').sum()
total         = len(q5_result)
pct_disadv    = round(female_disadv / total * 100, 1)
worst         = q5_result.iloc[0]

print(f"""
FINDING 2 — GENDER GAP IN YOUTH UNEMPLOYMENT
---------------------------------------------
In {female_disadv} out of {total} AU member states ({pct_disadv}%),
young women face higher unemployment than young men.
The largest gap is in {worst['country_code']} ({worst['au_region']})
where female youth unemployment exceeds male by
{abs(worst['gender_gap'])} percentage points.
""")

# Finding 3: Data completeness warning
poor = q6_result[q6_result['completeness_pct'] < 50]
print(f"""
FINDING 3 — DATA QUALITY WARNING
---------------------------------
{len(poor)} region-indicator combinations have less than
50% data completeness. This primarily affects
girls_secondary_enrol across Central and West Africa.

Recommendation: AU member states in these regions
should be engaged to improve national data collection
and reporting to UNESCO and World Bank systems.
""")

# Continental trend summary
trend_2015 = q2_result[
    (q2_result['indicator'] == 'women_in_parliament') &
    (q2_result['year'] == 2015)
]['continental_avg'].values[0]

trend_latest = q2_result[
    (q2_result['indicator'] == 'women_in_parliament') &
    (q2_result['year'] == q2_result['year'].max())
]['continental_avg'].values[0]

change = round(trend_latest - trend_2015, 1)
direction = "increased" if change > 0 else "decreased"

print(f"""
FINDING 4 — CONTINENTAL TREND: WOMEN IN PARLIAMENT
----------------------------------------------------
The continental average for women in parliament has
{direction} by {abs(change)} percentage points
between 2015 and {q2_result['year'].max()}
(from {trend_2015}% to {trend_latest}%).
""")

print("=" * 60)
print("✅ Analysis complete — ready for Power BI dashboard")
print("=" * 60)


AFRICA GENDER & YOUTH INDICATORS — KEY FINDINGS
AU WGYD M&E Data Pipeline | Teresiah Njoroge | Feb 2026

FINDING 1 — WOMEN IN PARLIAMENT
--------------------------------
The continent shows significant variation in female
political representation across AU member states.
The leading country significantly outperforms the
continental average, while the bottom performers
remain far below the AU's gender parity targets.

  Top performer:    RWA (East Africa) — 61.25%
  Lowest performer: NGA (West Africa) — 3.91%

FINDING 2 — GENDER GAP IN YOUTH UNEMPLOYMENT
---------------------------------------------
In 36 out of 52 AU member states (69.2%),
young women face higher unemployment than young men.
The largest gap is in LBY (Other)
where female youth unemployment exceeds male by
26.78 percentage points.


FINDING 3 — DATA QUALITY WARNING
---------------------------------
5 region-indicator combinations have less than
50% data completeness. This primarily affects
girls_secondary_enrol across C

## ✅ Notebook 02 Complete

### What We Built
This notebook loaded 2,430 rows of cleaned gender and youth
indicator data into a SQL database and answered 6 key
M&E questions using structured SQL queries.

### Files Used
- **Input:** `01_data/cleaned/` — 5 cleaned CSV files
- **Database:** SQLite in-memory database (rebuilt each run)

### Queries Summary
| Query | Question Answered |
|---|---|
| Q1 | Latest value per country per indicator |
| Q2 | Continental average trend 2015–2023 |
| Q3 | Regional averages — who leads, who lags |
| Q4 | Top 5 and bottom 5 — women in parliament |
| Q5 | Gender gap in youth unemployment by country |
| Q6 | Data completeness report by region |

### Key Takeaway for Stakeholders
- East Africa leads in female political representation
- North Africa has the largest gender gap in youth unemployment
- Girls secondary enrolment data is critically incomplete
  in Central and West Africa — urgent data collection needed

### Next Steps
| Step | Task |
|---|---|
| Notebook 03 | Excel export for non-technical staff |
| Power BI | Connect cleaned CSVs and build dashboard |
| GitHub | Push all files to repository |
